# Preprocessing danych

In [ ]:
import csv

In [35]:
money = 100000
debt = 250000

print("cheap: ", 0, " - ",money)
print("average: ", money, " - ", debt+money)
print("expensive: ", debt+money, " - ", "inf")

cheap:  0  -  100000
average:  100000  -  350000
expensive:  350000  -  inf


# Initial data viewing

In [24]:
# load data from csv
with open('zadanie_studenci/train_data.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)

columns_with_Samples = {}
for i in range(len(data[0])):
    columns_with_Samples[data[0][i]] = [data[1][i], data[2][i], data[3][i]]

for columns, vals in columns_with_Samples.items():
    print(columns,": ", vals)

SalePrice :  ['141592', '51327', '48672']
YearBuilt :  ['2006', '1985', '1985']
Size(sqf) :  ['814', '587', '587']
Floor :  ['3', '8', '6']
HallwayType :  ['terraced', 'corridor', 'corridor']
HeatingType :  ['individual_heating', 'individual_heating', 'individual_heating']
AptManageType :  ['management_in_trust', 'self_management', 'self_management']
N_Parkinglot(Ground) :  ['111.0', '80.0', '80.0']
N_Parkinglot(Basement) :  ['184.0', '76.0', '76.0']
TimeToBusStop :  ['5min~10min', '0~5min', '0~5min']
TimeToSubway :  ['10min~15min', '5min~10min', '5min~10min']
N_manager :  ['3.0', '2.0', '2.0']
N_elevators :  ['0.0', '2.0', '2.0']
SubwayStation :  ['Kyungbuk_uni_hospital', 'Daegu', 'Daegu']
N_FacilitiesInApt :  ['5', '3', '3']
N_FacilitiesNearBy(Total) :  ['6.0', '12.0', '12.0']
N_SchoolNearBy(Total) :  ['9.0', '4.0', '4.0']


In [19]:
idx = data[0].index("N_Parkinglot(Ground)")
print("N_Parkinglot(Ground): ", data[1][idx], data[2][idx], data[3][idx])

print(data[1][idx].split(".")[1] != '0')
print(data[1][idx].isdigit())

N_Parkinglot(Ground):  111.0 80.0 80.0
False
False


In [22]:
columns_with_types = {}

for i in range(len(data[0])):
    column = data[0][i]
    columns_vals = [data[x][i] for x in range(1, len(data))]
    # check for nones
    if None in columns_vals:
        print(column, " has None values")
        continue
    has_comma = ['.' in x for x in columns_vals]
    types = []
    for n, x in enumerate(columns_vals):
        if has_comma[n]:
            if x.split(".")[1] == '0':
                types.append("int")
            else:
                types.append("float")
        elif x.isdigit():
            types.append("int")
        else:
            types.append("string")
    #get unique types
    unique_types = set(types)
    if len(unique_types) > 1:
        print(column, " has mixed types")
        continue
    else:
        type_ = unique_types.pop()
        print(column, " has type: ", type_)
        columns_with_Samples[column].append(type_)

SalePrice  has type:  int
YearBuilt  has type:  int
Size(sqf)  has type:  int
Floor  has type:  int
HallwayType  has type:  string
HeatingType  has type:  string
AptManageType  has type:  string
N_Parkinglot(Ground)  has type:  int
N_Parkinglot(Basement)  has type:  int
TimeToBusStop  has type:  string
TimeToSubway  has type:  string
N_manager  has type:  int
N_elevators  has type:  int
SubwayStation  has type:  string
N_FacilitiesInApt  has type:  int
N_FacilitiesNearBy(Total)  has type:  int
N_SchoolNearBy(Total)  has type:  int


In [23]:
numerical_columns = []
categorical_columns = []

for column, vals in columns_with_Samples.items():
    if vals[-1] == "int" or vals[-1] == "float":
        numerical_columns.append(column)
    else:
        categorical_columns.append(column)

print("Numerical columns: ", numerical_columns)
print("Categorical columns: ", categorical_columns)

Numerical columns:  ['SalePrice', 'YearBuilt', 'Size(sqf)', 'Floor', 'N_Parkinglot(Ground)', 'N_Parkinglot(Basement)', 'N_manager', 'N_elevators', 'N_FacilitiesInApt', 'N_FacilitiesNearBy(Total)', 'N_SchoolNearBy(Total)']
Categorical columns:  ['HallwayType', 'HeatingType', 'AptManageType', 'TimeToBusStop', 'TimeToSubway', 'SubwayStation']


In [30]:
for colm in categorical_columns:
    idx = data[0].index(colm)
    col_vals = [data[x][idx] for x in range(1, len(data))]
    unique_vals = set(col_vals)
    counts = {}
    for val in col_vals:
        if val not in counts:
            counts[val] = 1
        else:
            counts[val] += 1
    print(colm, " has unique values: ", counts)

HallwayType  has unique values:  {'terraced': 2485, 'corridor': 445, 'mixed': 1194}
HeatingType  has unique values:  {'individual_heating': 3938, 'central_heating': 186}
AptManageType  has unique values:  {'management_in_trust': 3869, 'self_management': 255}
TimeToBusStop  has unique values:  {'5min~10min': 938, '0~5min': 3148, '10min~15min': 38}
TimeToSubway  has unique values:  {'10min~15min': 526, '5min~10min': 792, '0-5min': 1960, '15min~20min': 682, 'no_bus_stop_nearby': 164}
SubwayStation  has unique values:  {'Kyungbuk_uni_hospital': 1125, 'Daegu': 62, 'Sin-nam': 468, 'Myung-duk': 1065, 'Chil-sung-market': 85, 'Bangoge': 512, 'Banwoldang': 531, 'no_subway_nearby': 276}


In [37]:
targets = [int(data[x][0]) for x in range(1, len(data))]
categorized_targets = []
max_target = 0
min_target = 1000000000

for target in targets:
    if target > max_target:
        max_target = target
    if target < min_target:
        min_target = target


    if target < money:
        categorized_targets.append('cheap')
    elif target < debt+money:
        categorized_targets.append('average')
    else:
        categorized_targets.append('expensive')

target_counts = {}

for target in categorized_targets:
    if target not in target_counts:
        target_counts[target] = 1
    else:
        target_counts[target] += 1

print("Target counts: ", target_counts)
print("Max target: ", max_target)
print("Min target: ", min_target)

Target counts:  {'average': 2998, 'cheap': 556, 'expensive': 570}
Max target:  585840
Min target:  34070


# With this knowledge we start data preprocessing

- Plan is like this: for now to leave the numerical vals as they are and to just convert the data to ints
- for coulumns TimeToBusStop, TimeToSubway do index encoding, as is thinkt that actually it will be good for the model to have a correlation that 10-15 minutes is closer to 5-10mins then 0-5 mins
- 

In [ ]:
numerical_columns = []
categorical_columns = []